<h1><center> Protein classification using Random Forest </center></h1>

Using the new dataset with environment features

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import os
import sh
import sys
import math
import time
import random

from tqdm import tqdm
from pprint import pprint
from pathlib import Path

from sklearn.externals import joblib
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, auc
from sklearn.metrics import f1_score, classification_report


import seaborn as sns; sns.set(style='ticks', font='serif')

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

In [2]:
data_dir = "/home/lab6nb/random_forests/final_dataset/"

Make header consistent throughout the files by running the following command in the terminal

```
var='"","cmi.m.Var1","cmi.m.Var2","cmi.m$value","ecmi.m$value","cc.m$value","ecc.m$value","cp.m$value","ecp.m$value","cp1.m$value","ecp1.m$value","cp2.m$value","ecp2.m$value","hcm.m$value","ehcm.m$value","rsa.m$value","ersa.m$value","scm.m$value","escm.m$value","ssp.m$value","essp.m$value","inf.m$value"'
for i in $( ls *.csv ); do
    sed -i "1s/.*/$var/" $i
done
```

In [3]:
df = pd.read_csv(data_dir + "1a2k_inf_18_df.csv", engine= 'python')

df_x = (df[["cmi.m.value","cc.m.value","cp.m.value","cp1.m.value", "cp2.m.value","hcm.m.value","rsa.m.value",
            "scm.m.value","ssp.m.value", "ecc.m.value", "ecmi.m.value", "ecp.m.value", "ecp1.m.value", "ecp2.m.value", "ehcm.m.value", "ersa.m.value", "escm.m.value", "essp.m.value"]])

df_y = (df[["inf.m.value"]])

df_x.shape, df_y[df_y["inf.m.value"] == 1].shape

((24304, 18), (117, 1))

In [4]:
file_list = glob.glob(data_dir + "*.csv")

for file_name in file_list:
    
        df_temp = pd.read_csv(file_name)
        df_xn = (df_temp[["cmi.m.value","cc.m.value","cp.m.value","cp1.m.value", "cp2.m.value","hcm.m.value","rsa.m.value",
            "scm.m.value","ssp.m.value", "ecc.m.value", "ecmi.m.value", "ecp.m.value", "ecp1.m.value", "ecp2.m.value", "ehcm.m.value", "ersa.m.value", "escm.m.value", "essp.m.value"]])
        df_yn = (df_temp[["inf.m.value"]])
        df_x = df_x.append(df_xn, ignore_index=True)
        df_y = df_y.append(df_yn, ignore_index=True)
        print("'%-40s' file read with %i True" %(file_name, df_temp[df_temp["inf.m.value"] == 1].shape[0]))

'/home/lab6nb/random_forests/final_dataset/1ib1_inf_18_df.csv' file read with 140 True
'/home/lab6nb/random_forests/final_dataset/1oph_inf_18_df.csv' file read with 68 True
'/home/lab6nb/random_forests/final_dataset/1xqs_inf_18_df.csv' file read with 190 True
'/home/lab6nb/random_forests/final_dataset/1s1q_inf_18_df.csv' file read with 106 True
'/home/lab6nb/random_forests/final_dataset/1h9d_inf_18_df.csv' file read with 168 True
'/home/lab6nb/random_forests/final_dataset/3aaa_inf_18_df.csv' file read with 413 True
'/home/lab6nb/random_forests/final_dataset/1rv6_inf_18_df.csv' file read with 44 True
'/home/lab6nb/random_forests/final_dataset/2c0l_inf_18_df.csv' file read with 98 True
'/home/lab6nb/random_forests/final_dataset/3cph_inf_18_df.csv' file read with 95 True
'/home/lab6nb/random_forests/final_dataset/1i4d_inf_18_df.csv' file read with 59 True
'/home/lab6nb/random_forests/final_dataset/3k75_inf_18_df.csv' file read with 55 True
'/home/lab6nb/random_forests/final_dataset/1gcq_i

In [5]:
df_x

,cmi.m.value,cc.m.value,cp.m.value,cp1.m.value,cp2.m.value,hcm.m.value,rsa.m.value,scm.m.value,ssp.m.value,ecc.m.value,ecmi.m.value,ecp.m.value,ecp1.m.value,ecp2.m.value,ehcm.m.value,ersa.m.value,escm.m.value,essp.m.value
0,0.747425,0.642105,0.380208,0.970297,0.649485,0.842105,0.875000,0.865854,0.6,0.162969,0.495677,0.098793,0.294073,0.200047,0.140610,0.385793,0.249866,0.252830
1,0.460760,0.615789,0.276042,0.712871,0.855670,0.347368,0.250000,0.932662,0.4,0.288259,0.512112,0.211888,0.352209,0.245423,0.252022,0.369684,0.387998,0.318868
2,0.546943,0.642105,0.380208,0.970297,0.649485,0.842105,0.500000,0.865854,0.4,0.346121,0.413206,0.249696,0.351078,0.243766,0.270865,0.299130,0.457416,0.333962
3,0.181230,0.652632,0.231771,0.118812,0.721649,0.331579,0.000000,0.723754,0.4,0.320334,0.283853,0.261597,0.213249,0.208333,0.290805,0.180735,0.451954,0.313208
4,0.036292,0.310526,0.437500,0.346535,0.329897,1.000000,0.250000,0.865854,0.4,0.301244,0.207751,0.256929,0.234178,0.231929,0.305807,0.111791,0.444264,0.313208
5,0.233629,0.615789,0.244792,0.801980,0.876289,0.357895,0.000000,0.977731,0.4,0.305891,0.175850,0.257422,0.213500,0.332860,0.289781,0.049936,0.454101,0.313208
6,0.189211,0.615789,0.276042,0.712871,0.855670,0.347368,0.000000,0.932662,0.4,0.328170,0.154038,0.233586,0.255421,0.443734,0.256412,0.019652,0.449719,0.313208
7,0.088440,0.615789,0.244792,0.801980,0.876289,0.357895,0.000000,0.977731,0.4,0.336735,0.102566,0.228589,0.320847,0.396149,0.256119,0.000000,0.388029,0.360377
8,0.017652,0.615789,0.276042,0.712871,0.855670,0.347368,0.000000,0.932662,0.4,0.334685,0.049067,0.228885,0.308466,0.364189,0.247631,0.035438,0.369926,0.426415
9,0.009648,0.615789,0.424479,0.960396,0.969072,0.505263,0.000000,0.619300,0.6,0.357009,0.021868,0.286452,0.346616,0.211095,0.303136,0.132088,0.303545,0.501887


In [6]:
df_x1 = df_x[df_y['inf.m.value']==0] # Dataframe x-axis with all zeros
df_y1 = df_y[df_y['inf.m.value']==0] # Dataframe y-axis with all zeros

df_x2 = df_x[df_y['inf.m.value']==1] # Dataframe x-axis with all ones
df_y2 = df_y[df_y['inf.m.value']==1] # Dataframe y-axis with all ones

In [7]:
df_x1.shape, df_y1.shape, df_x2.shape, df_y2.shape

((2024466, 18), (2024466, 1), (5189, 18), (5189, 1))

In [8]:
# Choose random indices from df_x1.index with length = 5*length of df_x2

rind = np.random.choice(np.array(df_x1.index), size=5*len(df_x2))

In [9]:
rind[1]

1087657

In [10]:
df_x1.loc[rind]

,cmi.m.value,cc.m.value,cp.m.value,cp1.m.value,cp2.m.value,hcm.m.value,rsa.m.value,scm.m.value,ssp.m.value,ecc.m.value,ecmi.m.value,ecp.m.value,ecp1.m.value,ecp2.m.value,ehcm.m.value,ersa.m.value,escm.m.value,essp.m.value
1349855,0.011102,0.405263,0.273438,0.336634,0.350515,0.510526,0.000000,0.940085,0.1,0.383168,0.165535,0.571064,0.460457,0.505420,0.662912,0.060922,0.866626,0.000000
1087657,0.155441,0.500000,0.401042,0.752475,0.000000,0.800000,0.093750,0.573700,0.5,0.392601,0.431736,0.590576,0.544532,0.550641,0.748872,0.365719,0.659033,0.769811
1551562,0.038375,0.494737,0.476562,0.316832,0.567010,1.000000,0.000000,0.917885,0.3,0.565343,0.154653,0.746485,0.652453,0.672705,0.729007,0.182963,0.953806,0.586792
832648,0.054034,0.373684,0.356771,0.722772,0.701031,0.615789,0.281250,0.934191,0.6,0.209543,0.134576,0.279401,0.289525,0.257742,0.372619,0.141054,0.290323,0.507547
867907,0.069107,0.642105,0.338542,0.366337,0.907216,0.200000,0.000000,0.926038,0.3,0.690153,0.108392,0.665610,0.526711,0.617934,0.439868,0.104677,0.674300,0.400000
1971076,0.385236,0.836842,0.669271,0.089109,0.051546,0.810526,0.656250,0.858431,0.6,0.620394,0.262451,0.490514,0.232584,0.353710,0.571328,0.630983,0.645799,0.701887
473205,0.317978,0.484211,0.315104,0.564356,0.783505,0.794737,0.000000,0.865854,0.5,0.594572,0.425534,0.643197,0.548401,0.691873,0.702331,0.338503,0.797114,0.800000
806946,0.181526,0.500000,0.505208,0.702970,0.845361,0.989474,0.093750,0.803156,0.5,0.541015,0.202939,0.684061,0.564696,0.643566,0.734095,0.200445,0.755686,0.800000
637405,0.000000,0.515789,0.330729,0.039604,0.041237,0.878947,0.000000,0.492577,0.4,0.662973,0.041986,0.664823,0.465273,0.473639,0.693108,0.023968,0.613207,0.587736
1205772,0.113351,0.500000,0.367188,0.940594,0.577320,0.768421,0.078125,0.458972,0.3,0.504782,0.181921,0.694459,0.619201,0.689943,0.726465,0.159322,0.710487,0.543396


In [11]:
df_x3 = df_x1.loc[rind]
df_y3 = df_y1.loc[rind]

In [12]:
df_xc = pd.concat([df_x3, df_x2], ignore_index=True)
df_yc = pd.concat([df_y3, df_y2], ignore_index=True)

In [13]:
df_xc.shape, df_yc.shape

((31134, 18), (31134, 1))

In [14]:
x = (df_xc.as_matrix())
y = (df_yc.as_matrix())

/home/lab6nb/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/home/lab6nb/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [15]:
# for i in tqdm(range(len(x))):
#     x[i,0] = dict_var12[x[i,0]]
#     x[i,1] = dict_var12[x[i,1]]

In [16]:
scaler = StandardScaler().fit(x)
rescaledX = scaler.transform(x)

In [17]:
y = y.ravel()

In [18]:
X_train, X_test, y_train, y_test = train_test_split(rescaledX, y, test_size=0.25, random_state=42)

In [19]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(200, 2000, 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

pprint(random_grid)

lengths = [len(lst) for lst in random_grid.values()]

combs=1
for i in range(len(lengths)):
    combs *= lengths[i]
print('Total number of combinations: {:16d}'.format(combs))  


{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}
Total number of combinations:             4320


In [20]:
# Use the random grid to search for best hyperparameters

# Base model to tune
rf = RandomForestClassifier(class_weight='balanced')

# Random search of parameters, using 5 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,
                               scoring='f1', n_iter = 100, cv = 5, verbose=1, 
                               random_state=42, n_jobs = -1)

In [ ]:
# Fit the random search model
time_start = time.time()
rf_random.fit(X_train, y_train)
e = int(time.time() - time_start) # elapsed time
print("Time taken in fitting: {:02d}h:{:02d}m:{:5.2f}s".format(e//3600, e % 3600 // 60, e % 60))

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
/home/lab6nb/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/home/lab6nb/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  3.7min
/home/lab6nb/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a me

In [ ]:
joblib.dump(rf_random, 'RF_random_grid_model_with_env_model_leaving_3i4r.sav')

In [ ]:
rf_random.best_params_

In [ ]:
best_random = rf_random.best_estimator_
random_accuracy = best_random.score(X_test, y_test)
print("Best Random model Accuracy: %.2f%%" % (random_accuracy * 100.0))

In [ ]:
y_pred = best_random.predict(X_test)

Further fine-tuning near the best fitted parameters

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

cm = confusion_matrix(y_test, y_pred)

fig, ax = plt.subplots(figsize=(10, 8))
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
cmap = sns.cubehelix_palette(light=0.98, as_cmap=True)
unsuv_cm = pd.DataFrame(cm)
sns.heatmap(unsuv_cm, annot=True,annot_kws={"size": 16}, cmap=cmap, fmt='d')# font size
plt.xlabel("Predicted Class", fontsize=20, labelpad=20)
plt.ylabel("True Class", fontsize=20, labelpad=20)
plt.show()

In [ ]:
fig.savefig('conf_matrix_leaving_3i4r.svg', format= 'svg')
fig.savefig('conf_matrix_leaving_3i4r.eps', format= 'eps')

In [ ]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True, False],
    'max_depth': [10, 20, 40],
    'max_features': ['sqrt'],
    'min_samples_leaf': [2, 4,6],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [1000, 1200, 1400]
}

In [ ]:
# Create a base model
rf = RandomForestClassifier(class_weight='balanced')

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, scoring='f1',
                           cv = 5, n_jobs = -1, verbose = 1)

In [ ]:
time_start = time.time()
grid_search.fit(X_train, y_train);
e = int(time.time() - time_start) # elapsed time

print("Fitting Time: {:02d}h:{:02d}m:{:5.2f}s".format(e//3600, e % 3600 // 60, e % 60))

In [ ]:
# joblib.dump(grid_search, 'RF_grid_search_model.sav')

In [ ]:
grid_search.best_params_

In [ ]:
best_grid = grid_search.best_estimator_

In [ ]:
final_model = grid_search.best_estimator_

In [ ]:
print('Final Model Parameters:\n')
pprint(final_model.get_params())
print('\n')
grid_final_accuracy = final_model.score(X_test, y_test)
print("Grid Final Accuracy: %.2f%%" % (grid_final_accuracy * 100.0))

In [ ]:
y_pred = final_model.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, y_pred)

fig, ax = plt.subplots(figsize=(10, 8))
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
cmap = sns.cubehelix_palette(light=0.98, as_cmap=True)
unsuv_cm = pd.DataFrame(cm)
sns.heatmap(unsuv_cm, annot=True,annot_kws={"size": 16}, cmap=cmap, fmt='d')# font size
plt.xlabel("Predicted Class", fontsize=20, labelpad=20)
plt.ylabel("True Class", fontsize=20, labelpad=20)
plt.show()


In [ ]:
fig.savefig('conf_matrix_f1_leaving_3i4r_af_opt.svg', format= 'svg')
fig.savefig('conf_matrix_f1_leaving_3i4r_af_opt.eps', format= 'eps')

In [ ]:
print(classification_report(y_test, y_pred))

Feature Importance

In [ ]:
importances = final_model.feature_importances_
std = np.std([tree.feature_importances_ for tree in final_model.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]
print(final_model.feature_importances_)

In [ ]:
'''# Print the feature ranking
print("Feature ranking:")

for f in range(X_train.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))'''

In [ ]:
n_feature = 18
graph= plt.figure(figsize=(10,7))
plt.title("Feature importances")
plt.bar(range(X_train.shape[1])[0:n_feature], importances[indices[0:n_feature]],
       color="royalblue", yerr=std[indices[0:n_feature]], align="center")
plt.xticks(range(X_train.shape[1])[0:n_feature], indices[0:n_feature])
plt.xlim([-1, n_feature])
plt.show()
graph.savefig('feature_importace_run3.svg', format= 'svg')
graph.savefig('feature_importace_run3.eps', format= 'eps')


In [ ]:
fpr, tpr, threshold = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)

In [ ]:
auc_graph= plt.figure()
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()
auc_graph.savefig('auc_roc_run3.svg', format= 'svg')
auc_graph.savefig('auc_roc_run3.eps', format= 'eps')


#### Testing on "data_frames_01102018/3i4r_final_df.csv"

In [ ]:
df = pd.read_csv(data_dir + "3i4r_inf_18_df.csv")

df_x3i4r = (df[["cmi.m.value","cc.m.value","cp.m.value","cp1.m.value", "cp2.m.value","hcm.m.value","rsa.m.value",
            "scm.m.value","ssp.m.value", "ecc.m.value", "ecmi.m.value", "ecp.m.value", "ecp1.m.value", "ecp2.m.value", "ehcm.m.value", "ersa.m.value", "escm.m.value", "essp.m.value"]])

df_y3i4r = (df[["inf.m.value"]])

df_x3i4r.shape, df_y3i4r[df_y3i4r["inf.m.value"] == 1].shape

In [ ]:
xtest_3i4r = np.array(df_x3i4r)
ytest_3i4r = np.array(df_y3i4r)

In [ ]:
rsc_xtest_3i4r = scaler.transform(xtest_3i4r)

In [ ]:
ytest_3i4r = ytest_3i4r.ravel()

In [ ]:
ypred_3i4r = final_model.predict(rsc_xtest_3i4r)

In [ ]:
test_prob = final_model.predict_proba(rsc_xtest_3i4r)

In [ ]:
sns.distplot(test_prob[(ytest_3i4r == 0) & (ypred_3i4r == 1)][:,1])
sns.distplot(test_prob[(ytest_3i4r == 1) & (ypred_3i4r == 1)][:,1])
plt.show()

In [ ]:
threshold=0.53
predicted_proba = final_model.predict_proba(rsc_xtest_3i4r)
predicted = (predicted_proba [:,1] >= threshold).astype('int')

In [ ]:
predicted.shape, ytest_3i4r.shape

In [ ]:
cm = confusion_matrix(ytest_3i4r, predicted)#ypred_3i4r)

fig, ax = plt.subplots(figsize=(10, 8))
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
cmap = sns.cubehelix_palette(light=0.98, as_cmap=True)
unsuv_cm = pd.DataFrame(cm)
sns.heatmap(unsuv_cm, annot=True,annot_kws={"size": 16}, cmap=cmap, fmt='d')# font size
plt.xlabel("Predicted Class", fontsize=20, labelpad=20)
plt.ylabel("True Class", fontsize=20, labelpad=20)
plt.show()

In [ ]:
print(classification_report(ytest_3i4r, predicted))#ypred_3i4r))

In [ ]:
predicted = pd.DataFrame(predicted, columns=['prediction']).to_csv('prediction_3i4r.csv')